In [1]:
import numpy as np
import cv2

In [3]:
from scipy.spatial.transform import Rotation as R
posc = []
# data = np.load('../result/230422/0522test11.npz')
data = np.load('../result/230422/0424test4.npz')

OF = data['OF'] 
OF[:,:,:,0] = OF[:,:,:,0]*128 #光流大小*解析度

IMG = data['IMG']
POS = data['POS']
ORI = data['ORI']


#從旋轉矩陣和世界座標系位移，得到相機座標系的位移
for i in range(1,IMG.shape[0]):
    r = R.from_quat(ORI[i,:]).as_matrix()
    d_x = POS[i,0] - POS[i-1,0]
    d_z = POS[i,2] - POS[i-1,2]
    d_y = POS[i,1] - POS[i-1,1]
    d = [d_z, d_x, d_y]
    camera = np.matmul(r, d)
    posc.append(camera)

<class 'numpy.float32'>
[[ 9.94521920e-01  1.04528224e-01  1.10827851e-06]
 [ 3.43994169e-09 -1.06354012e-05  1.00000000e+00]
 [ 1.04528224e-01 -9.94521920e-01 -1.05774992e-05]]


In [4]:
#save image of raw image, optical flow map, and depth map if need.
frame = 0
import flowpy
while frame < OF.shape[0]:

    cv2.imwrite('/Users/hsinyu/Desktop/airsim2/raw{}.png'.format(frame), IMG[frame,:,:,:])

    #從r, theta轉成x,y 並用flowpy視覺化
    flow = np.stack((OF[frame,:,:,0]*np.cos(OF[frame,:,:,1]), OF[frame,:,:,0]*np.sin(OF[frame,:,:,1])),axis=-1)
    
    off = flowpy.flow_to_rgb(flow[:,:,:],flow_max_radius=1)
    off = cv2.cvtColor(off, cv2.COLOR_RGB2BGR)

    cv2.imwrite('/Users/hsinyu/Desktop/airsim2/of{}.png'.format(frame), off)
    
    frame += 1
    # cv2.waitKey(1)